In [4]:
! pip install kaggle && pip install torchinfo && pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.3/977.3 kB 31.3 MB/s eta 0:00:00


In [5]:
import torch
import torchvision
import os
from matplotlib import pyplot as plt


torch.__version__, torchvision.__version__

('2.5.1+cu121', '0.20.1+cu121')

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
from tqdm.auto import tqdm

def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc


def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(tqdm(data_loader, desc="Training  ", unit=" batch")):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    return train_loss, train_acc

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for X, y in tqdm(data_loader, desc="Evaluating", unit=' batch'):
            # Send data to GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        return test_loss, test_acc


def train(model, train_loader, test_loader, optimizer, loss_fn, accuracy_fn=accuracy_fn, epochs: int = 5, device=device):
    results = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': [],
    }
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
                                    model=model,
                                    data_loader=train_dataloader,
                                    loss_fn=loss_fn,
                                    optimizer=optimizer,
                                    accuracy_fn=accuracy_fn,
                                    device=device)
        test_loss, test_acc = test_step(
                                    model=model,
                                    data_loader=test_dataloader,
                                    loss_fn=loss_fn,
                                    accuracy_fn=accuracy_fn,
                                    device=device)

        print(f'Epoch {epoch} | Train Loss/Acc: {train_loss:.2f}/{train_acc} | Test Loss/Acc: {test_loss:.2f}/{test_acc}')

        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

    return results



def accuracy_fn(y_true, y_pred):
    """
    Computes the accuracy by comparing the true labels to the predicted labels.

    Args:
        y_true (torch.Tensor): Ground truth labels.
        y_pred (torch.Tensor): Predicted labels.

    Returns:
        float: Accuracy as a percentage.
    """
    correct = (y_true == y_pred).sum().item()
    total = y_true.size(0)
    return (correct / total) * 100

def get_all_classes(dataloader):
    """
    Extracts all unique classes from a PyTorch DataLoader.

    Args:
        dataloader (DataLoader): The DataLoader to extract classes from.

    Returns:
        list: A list of unique classes present in the DataLoader.
    """
    all_classes = []
    for _, labels in dataloader:
      all_classes.extend(labels.tolist())
    return list(set(all_classes))

In [11]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for MobileNet
])


In [12]:
from torchvision import datasets

dataset = datasets.FGVCAircraft(
    root="data_v2",
    download=True,
    transform=transform
)


100%|██████████| 2.75G/2.75G [00:23<00:00, 117MB/s] 


Extracting data_v2/fgvc-aircraft-2013b.tar.gz to data_v2


In [13]:
import os

dataset_path = "/content/data_v2/fgvc-aircraft-2013b/data/images"
yolo_path = "/content/dataset"

# Create YOLO structure
os.makedirs(f"{yolo_path}/images/train", exist_ok=True)
os.makedirs(f"{yolo_path}/images/val", exist_ok=True)
os.makedirs(f"{yolo_path}/images/test", exist_ok=True)
os.makedirs(f"{yolo_path}/labels/train", exist_ok=True)
os.makedirs(f"{yolo_path}/labels/val", exist_ok=True)
os.makedirs(f"{yolo_path}/labels/test", exist_ok=True)


In [14]:
import shutil

# Function to move images
def move_images(txt_file, dest_folder):
    with open(txt_file, "r") as f:
        images = f.read().splitlines()
        for img in images:
            img_path = os.path.join(dataset_path, img + ".jpg")  # Ensure correct file extension
            if os.path.exists(img_path):
                shutil.move(img_path, os.path.join(dest_folder, os.path.basename(img_path)))

# Move images to respective folders
move_images("/kaggle/working/data_v2/fgvc-aircraft-2013b/data/images_train.txt", f"{yolo_path}/images/train")
move_images("/kaggle/working/data_v2/fgvc-aircraft-2013b/data/images_val.txt", f"{yolo_path}/images/val")
move_images("/kaggle/working/data_v2/fgvc-aircraft-2013b/data/images_test.txt", f"{yolo_path}/images/test")


In [15]:
# Read variants.txt and create a mapping
variant_file = "/kaggle/working/data_v2/fgvc-aircraft-2013b/data/variants.txt"

with open(variant_file, "r") as f:
    variant_classes = [line.strip() for line in f.readlines()]

# Create a dictionary to map class names to indices
variant_to_id = {variant: idx for idx, variant in enumerate(variant_classes)}

print(variant_to_id)  # Example output: {'Boeing_747': 0, 'Airbus_A320': 1, ...}


{'707-320': 0, '727-200': 1, '737-200': 2, '737-300': 3, '737-400': 4, '737-500': 5, '737-600': 6, '737-700': 7, '737-800': 8, '737-900': 9, '747-100': 10, '747-200': 11, '747-300': 12, '747-400': 13, '757-200': 14, '757-300': 15, '767-200': 16, '767-300': 17, '767-400': 18, '777-200': 19, '777-300': 20, 'A300B4': 21, 'A310': 22, 'A318': 23, 'A319': 24, 'A320': 25, 'A321': 26, 'A330-200': 27, 'A330-300': 28, 'A340-200': 29, 'A340-300': 30, 'A340-500': 31, 'A340-600': 32, 'A380': 33, 'ATR-42': 34, 'ATR-72': 35, 'An-12': 36, 'BAE 146-200': 37, 'BAE 146-300': 38, 'BAE-125': 39, 'Beechcraft 1900': 40, 'Boeing 717': 41, 'C-130': 42, 'C-47': 43, 'CRJ-200': 44, 'CRJ-700': 45, 'CRJ-900': 46, 'Cessna 172': 47, 'Cessna 208': 48, 'Cessna 525': 49, 'Cessna 560': 50, 'Challenger 600': 51, 'DC-10': 52, 'DC-3': 53, 'DC-6': 54, 'DC-8': 55, 'DC-9-30': 56, 'DH-82': 57, 'DHC-1': 58, 'DHC-6': 59, 'DHC-8-100': 60, 'DHC-8-300': 61, 'DR-400': 62, 'Dornier 328': 63, 'E-170': 64, 'E-190': 65, 'E-195': 66, 'EMB

In [16]:
import os

# Paths
dataset_path = "/kaggle/working/data_v2/fgvc-aircraft-2013b/data/images"
train_labels_file = "/kaggle/working/data_v2/fgvc-aircraft-2013b/data/images_variant_train.txt"
train_dest = "/kaggle/working/dataset/labels/train"

os.makedirs(train_dest, exist_ok=True)

# Read image-to-label mappings
with open(train_labels_file, "r") as f:
    image_labels = [line.strip() for line in f.readlines()]

# Generate YOLO label files
for img_label in image_labels:
    parts = img_label.split(maxsplit=1)  # Split only at the first space

    if len(parts) < 2:
        print(f"Skipping malformed line: {img_label}")  # Debugging step
        continue

    img_name, class_name = parts
    class_id = variant_to_id.get(class_name, -1)  # Get class ID, default to -1 if not found

    if class_id == -1:
        print(f"Warning: Class '{class_name}' not found in variants.txt")
        continue

    # Define YOLO label file path
    label_file = os.path.join(train_dest, f"{img_name}.txt")

    # Dummy bounding box (since we lack bbox data)
    yolo_label = f"{class_id} 0.5 0.5 1.0 1.0\n"

    # Write label file
    with open(label_file, "w") as f:
        f.write(yolo_label)


In [17]:
# Paths for validation and test labels
val_labels_file = "/kaggle/working/data_v2/fgvc-aircraft-2013b/data/images_variant_val.txt"
test_labels_file = "/kaggle/working/data_v2/fgvc-aircraft-2013b/data/images_variant_test.txt"

val_dest = "/kaggle/working/dataset/labels/val"
test_dest = "/kaggle/working/dataset/labels/test"

os.makedirs(val_dest, exist_ok=True)
os.makedirs(test_dest, exist_ok=True)

# Function to generate labels
def generate_yolo_labels(txt_file, dest_folder):
    with open(txt_file, "r") as f:
        image_labels = [line.strip() for line in f.readlines()]

    for img_label in image_labels:
        parts = img_label.split(maxsplit=1)

        if len(parts) < 2:
            print(f"Skipping malformed line: {img_label}")
            continue

        img_name, class_name = parts
        class_id = variant_to_id.get(class_name, -1)

        if class_id == -1:
            print(f"Warning: Class '{class_name}' not found in variants.txt")
            continue

        # Define YOLO label file path
        label_file = os.path.join(dest_folder, f"{img_name}.txt")

        # Dummy bounding box (since we lack bbox data)
        yolo_label = f"{class_id} 0.5 0.5 1.0 1.0\n"

        with open(label_file, "w") as f:
            f.write(yolo_label)

# Generate labels for val and test
generate_yolo_labels(val_labels_file, val_dest)
generate_yolo_labels(test_labels_file, test_dest)


In [18]:
import shutil

# Paths for images
train_images_file = "/kaggle/working/data_v2/fgvc-aircraft-2013b/data/images_train.txt"
val_images_file =   "/kaggle/working/data_v2/fgvc-aircraft-2013b/data/images_val.txt"
test_images_file =  "/kaggle/working/data_v2/fgvc-aircraft-2013b/data/images_test.txt"

train_dest = "/kaggle/working/dataset/images/train"
val_dest = "/kaggle/working/dataset/images/val"
test_dest = "/kaggle/working/dataset/images/test"

os.makedirs(train_dest, exist_ok=True)
os.makedirs(val_dest, exist_ok=True)
os.makedirs(test_dest, exist_ok=True)

# Function to move images
def move_images(txt_file, dest_folder):
    with open(txt_file, "r") as f:
        images = f.read().splitlines()
        for img in images:
            img_path = os.path.join(dataset_path, img + ".jpg")  # Ensure correct file extension
            if os.path.exists(img_path):
                shutil.move(img_path, os.path.join(dest_folder, os.path.basename(img_path)))
            else:
                print(f"Warning: Image not found - {img_path}")

# Move images to respective folders
move_images(train_images_file, train_dest)
move_images(val_images_file, val_dest)
move_images(test_images_file, test_dest)


In [19]:
dataset_yaml = """path: /kaggle/working/dataset
train: images/train
val: images/val
test: images/test
nc: """ + str(len(variant_classes)) + """
names: """ + str(variant_classes)

with open("/kaggle/working//dataset/custom_dataset.yaml", "w") as f:
    f.write(dataset_yaml)


In [ ]:
# Get the classnames
class_names = dataset.classes
len(class_names), class_names

In [22]:
from ultralytics import YOLO

# Load model
model = YOLO("//kaggle/input/yolov8n/pytorch/default/1/best.pt")  # Adjust based on the correct checkpoint

# Train
model.train(data="/kaggle/working/dataset/custom_dataset.yaml", epochs=100, imgsz=640)


Ultralytics 8.3.101 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=//kaggle/input/yolov8n/pytorch/default/1/best.pt, data=/kaggle/working/dataset/custom_dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf

100%|██████████| 755k/755k [00:00<00:00, 45.1MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

100%|██████████| 5.35M/5.35M [00:00<00:00, 171MB/s]


AMP: checks passed ✅


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 105/105 [00:20<00:00,  5.20it/s]


                   all       3333       3333      0.818      0.798      0.857      0.857
               707-320         33         33      0.941      0.697      0.901      0.901
               727-200         34         34      0.852      0.882      0.895      0.895
               737-200         33         33      0.873      0.835      0.928      0.928
               737-300         33         33      0.616      0.303      0.624      0.624
               737-400         34         34      0.672      0.676      0.795      0.795
               737-500         33         33      0.583      0.636      0.736      0.736
               737-600         33         33      0.818      0.818      0.923      0.923
               737-700         34         34      0.569      0.794      0.752      0.752
               737-800         33         33      0.591      0.727      0.707      0.707
               737-900         33         33      0.902      0.834        0.9        0.9
               747-10

/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e366eea5f00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018, 